In [3]:
import cv2
import numpy as np
import dlib
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
for i in range(100):
    face_path = './dataset/train/img/%s.png'%i
    face_origin = cv2.imread(face_path)
    face_copy = face_origin.copy()
    face_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)
    
    detector = dlib.get_frontal_face_detector()
    dlib_rects = detector(face_rgb, 1)
    
    img_how_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)
    
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    
    globals()['height'+str(i)] = face_origin.shape[0]
    
    globals()['img'+str(i)] = []
    
    for dlib_rect in dlib_rects:
        points = predictor(face_rgb, dlib_rect)
        list_points = list(map(lambda p: [p.x, p.y], points.parts()))
        globals()['img'+str(i)].append(list_points)
        globals()['img'+str(i)] = np.array(globals()['img'+str(i)])

In [5]:
face_origin.shape

(1080, 800, 3)

In [6]:
for i in range(100):
    land = np.zeros((68,2))
    
    for j in range(68):
        land[j][0] = globals()['img'+str(i)][0][j][0]
        land[j][1] = globals()['img'+str(i)][0][j][1]
        
    x = land[:,0]
    y = land[:,1] * (-1) + globals()['height'+str(i)]
    
    x -= ((min(x) + max(x)) / 2)
    y -= ((min(y) + max(y)) / 2)
    
    norm_x = (x - min(x)) / (max(y)-min(y))
    norm_y = (y - min(y)) / (max(y)-min(y))
    
    globals()['norm'+str(i)] = np.column_stack((norm_x, norm_y))

In [7]:
# xyxyxyxyxyxyxy(0~16)
for i in range(100):
    globals()['norm'+str(i)] = globals()['norm'+str(i)][48:]
    globals()['norm'+str(i)] = globals()['norm'+str(i)].flatten()

In [8]:
train_x = np.zeros((100, len(norm0)))
for i in range(100):
    for j in range(len(norm0)):
        train_x[i][j] = globals()['norm'+str(i)][j]   

In [9]:
for i in range(100):
    face_path = './dataset/test/img/%s.png'%i
    face_origin = cv2.imread(face_path)
    face_copy = face_origin.copy()
    face_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)
    
    detector = dlib.get_frontal_face_detector()
    dlib_rects = detector(face_rgb, 1)
    
    img_how_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)
    
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    
    globals()['height'+str(i)] = face_origin.shape[0]
    
    globals()['img'+str(i)] = []
    
    for dlib_rect in dlib_rects:
        points = predictor(face_rgb, dlib_rect)
        list_points = list(map(lambda p: [p.x, p.y], points.parts()))
        globals()['img'+str(i)].append(list_points)
        globals()['img'+str(i)] = np.array(globals()['img'+str(i)])

In [10]:
for i in range(100):
    land = np.zeros((68,2))
    
    for j in range(68):
        land[j][0] = globals()['img'+str(i)][0][j][0]
        land[j][1] = globals()['img'+str(i)][0][j][1]
        
    x = land[:,0]
    y = land[:,1] * (-1) + globals()['height'+str(i)]
    
    x -= ((min(x) + max(x)) / 2)
    y -= ((min(y) + max(y)) / 2)
    
    norm_x = (x - min(x)) / (max(y)-min(y))
    norm_y = (y - min(y)) / (max(y)-min(y))
    
    globals()['norm'+str(i)] = np.column_stack((norm_x, norm_y))

In [11]:
# xyxyxyxyxyxyxy(0~16)
for i in range(100):
    globals()['norm'+str(i)] = globals()['norm'+str(i)][48:]
    globals()['norm'+str(i)] = globals()['norm'+str(i)].flatten()

In [12]:
test_x = np.zeros((100, len(norm0)))
for i in range(100):
    for j in range(len(norm0)):
        test_x[i][j] = globals()['norm'+str(i)][j]   

In [13]:
train_y = pd.read_csv('./dataset/train/train.csv')
train_y = train_y.iloc[:, 30:]
test_y = pd.read_csv('./dataset/test/test.csv')
test_y = test_y.iloc[:, 30:]

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

model = LinearRegression()
model.fit(train_x, train_y)
y_pred = model.predict(train_x)
mae = mean_absolute_error(train_y, y_pred)

mae

0.05257362026002028

In [15]:
y_pred = model.predict(test_x)
mae = mean_absolute_error(test_y, y_pred)

mae

0.08244349782035656

In [16]:
model.intercept_, model.coef_

(array([ 0.03869733, -3.25134115,  0.89562899, -0.27390142]),
 array([[ -9.35192753,   2.16605962,  -3.37446076,   9.40573991,
          -0.45523167,  -3.50652067,  -3.99417862,  -1.36707589,
         -11.36611792,  -3.53875793,   4.31653427, -10.90191997,
          -3.50658077,  -2.83113599,   3.42342126,   6.20518473,
           1.58533598,  -7.11715606,   8.31964878,   0.61559654,
           3.59551994,   3.80245788,  -6.75907906,   2.0318357 ,
           3.86829357,  12.19380031,  -2.59931337,  -4.4944092 ,
          -3.362048  ,  -4.10408863,  10.24772442,   4.10532502,
           3.17992335,  -0.87085905,  -0.69082871,   4.8888689 ,
           0.80252249, -10.011287  ,   3.14211236,   2.98937134],
        [ -4.19275927, -10.84053572,   5.81809872,  18.54112218,
          -0.05264847, -10.04848617,  12.14676148,  15.05816031,
          -0.15636768,  -9.15374742,  -1.54608811,  -7.98881113,
          -3.71401728,   5.52842713,  -2.83327542,  -4.85962071,
           1.81072076,   6.

In [20]:
# pd.set_option('display.max_rows', None)

In [17]:
prediction = pd.DataFrame(y_pred)
prediction.columns = ["mouthSize","mouthPosition","mouthPronounced","lipSize"]
prediction

,mouthSize,mouthPosition,mouthPronounced,lipSize
0,0.190955,0.505558,0.478015,0.562302
1,0.307440,0.824935,0.620684,0.758519
2,0.290873,0.631597,0.386745,0.478125
3,0.423384,0.175056,0.451758,0.323483
4,0.554192,0.824096,0.548247,0.521639
...,...,...,...,...
95,0.541755,0.321628,0.370032,0.560508
96,0.614455,0.563094,0.306541,0.788298
97,0.473311,0.406101,0.703929,0.529626
98,0.642149,0.395866,0.380381,0.607779


In [18]:
test_y

,mouthSize,mouthPosition,mouthPronounced,lipsSize
0,0.241427,0.614988,0.780424,0.520251
1,0.298667,0.704906,0.733134,0.742770
2,0.228592,0.651212,0.317283,0.440955
3,0.481674,0.214611,0.633301,0.222423
4,0.676315,0.793210,0.581524,0.521908
...,...,...,...,...
95,0.549648,0.204712,0.569278,0.322924
96,0.617910,0.485857,0.463260,0.680530
97,0.445559,0.314909,0.787422,0.434210
98,0.660861,0.420872,0.230623,0.670635


In [19]:
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(test_y, prediction)**0.5
print(rmse)

0.11102132902313668
